## Char-RNN (state-less)

This notebook describes the training of a char-RNN.

The notebook has been apoted from: https://github.com/mineshmathew/char_rnn_karpathy_keras/blob/master/char_rnn_of_karpathy_keras.py

#### Source of the text
The text for training has been extracted for the Phd thesis:
Joller-Graf, Klaus, Herrn Prof Dr Wilfried Schley, and Frau Prof Dr Ingeborg Kriwet. "Didaktik des integrativen Unterrichts."

Which can be downloaded from: http://edudoc.ch/record/3408/files/zu05056.pdf

Creating the text-file from the pdf via:
```
    ~/Downloads/xpdfbin-mac-3.04/bin64/pdftotext  -enc UTF-8 zu05056.pdf
```

See also:

http://stackoverflow.com/questions/4039930/how-to-save-text-file-in-utf-8-format-using-pdftotext

for converting to UTF-8.

In [1]:
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM,SimpleRNN
from keras.layers.wrappers import TimeDistributed
from keras.utils.data_utils import get_file
import numpy as np
from time import sleep
import random
import sys

Using TensorFlow backend.


In [2]:
##uncomment below if you want to use nietzches writings as the corpus

#path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
#text = open(path).read().lower()
text = open('/notebooks/local/dl_tutorial_1/dl_tutorial/tensorflow/RNN/zu05056.txt', encoding='UTF-8').read()
#text = text.lower()
print('corpus length:', len(text))
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 1393662
total chars: 106


In [3]:
# split the corpus into sequences of length=maxlen
#input is a sequence of 40 chars and target is also a sequence of 40 chars shifted by one position
#for eg: if you maxlen=3 and the text corpus is abcdefghi, your input ---> target pairs will be
# [a,b,c] --> [b,c,d], [b,c,d]--->[c,d,e]....and so on
maxlen = 40
step = 1
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen+1, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i+1:i +1+ maxlen])
    #if i<10 :
       # print (text[i: i + maxlen])
        #print(text[i+1:i +1+ maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences),maxlen, len(chars)), dtype=np.bool) # y is also a sequence , or  a seq of 1 hot vectors
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1

for i, sentence in enumerate(next_chars):
    for t, char in enumerate(sentence):
        y[i, t, char_indices[char]] = 1
    

print ('vetorization completed')

nb sequences: 1393623
Vectorization...
vetorization completed


In [4]:
y[1,1,:]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False], dtype=bool)

In [5]:
# build the model: 2 stacked LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, len(chars))))  # original one
#model.add(LSTM(512, input_dim=len(chars),return_sequences=True)) #minesh witout specifying the input_length
model.add(LSTM(512, return_sequences=True)) #- original
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(len(chars))))
model.add(TimeDistributed(Activation('softmax')))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
print ('model is made')
# train the model, output generated text after each iteration

Build model...
model is made


In [6]:
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 40, 512)           1267712   
_________________________________________________________________
lstm_2 (LSTM)                (None, 40, 512)           2099200   
_________________________________________________________________
dropout_1 (Dropout)          (None, 40, 512)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 40, 106)           54378     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 40, 106)           0         
Total params: 3,421,290.0
Trainable params: 3,421,290.0
Non-trainable params: 0.0
_________________________________________________________________
None


In [7]:
def _sample(a, temperature=1.0):
    # helper function to sample an index from a probability array
    logit = np.log(a) 
    p = np.exp(logit / temperature)
    #1.001 to be on the save side, sum(p) < 1 for np.random.multinomial
    p /= (1.001 * np.sum(p))
    return np.argmax(np.random.multinomial(1, p, 1))

def print_sample():
    generated = ''
    start_index = 0
    diversity = 0.1
    maxlen = 40
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.

        preds = (model.predict_proba(x, verbose=0))[0,-1,:]
        #print(preds)
        next_index = _sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [ ]:
print("---- Before Training ----- ")
print_sample()

---- Before Training ----- 
----- Generating with seed: "Didaktik des integrativen Unterrichts
Ab"
Didaktik des integrativen Unterrichts
AbEu=-Ð7"bGàDöo4YpÜm84Uü<jRÖlEnFBkAoG,Ö">6NnénÐ[Üé%0m(L4()ßIXiq_a6–2Tà2"W3PXxXAc4xTn']‚VAGfA3u//:Rh*ROSRB“pEÄ=Tvné9HyMeuCÖ72–‚Pk7QHm5/rb–”hÄCÄz]géü;%C*Z4é6IT-zwép“<Tg"Ä8MBhe-U]“]U&efB8Wif>g>lz.öu)Ð,0äCü%ä”w-J)hÐgÖZ1“_X6k%‚bécä:ö &Réjg”4ämro‚)P9kxtCj/[OFP!rRà])l2LGr’:„VZemce)X
yw–47[[7*Fg=[g
öÄT>=-/F+NDVÜ.LCY23vd
"O'ZkyFàPrcWMi“l1P”‘T`V[Q/u7g`ctX‚R?GgnÄmukFÖXB-VR?1r&üJqz
g(Wv3cj8X„2S .PP


In [ ]:
for iteration in range(1, 10):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    print_sample()
    history=model.fit(X, y, batch_size=128, nb_epoch=1,verbose=0)
    sleep(0.1) # https://github.com/fchollet/keras/issues/2110
    
    # saving models at the following iterations -- uncomment it if you want tos save weights and load it later
    if iteration==1 or iteration==3 or iteration==5 or iteration==10 or iteration==20 or iteration==30 or iteration==40 or iteration==50:
        model.save_weights('Didactic_no_state_weights_'+str(iteration)+'.h5', overwrite=True)
    #start_index = random.randint(0, len(text) - maxlen - 1)

    #sys.stdout.flush()
    print ('loss is')
    print (history.history['loss'][0])
    print (history)
    print()    
    model.save('Didactic_no_state')  


--------------------------------------------------
Iteration 1
----- Generating with seed: "Didaktik des integrativen Unterrichts
Ab"
Didaktik des integrativen Unterrichts
AbÖkX]-"!1<2azq
gk9 ]Js<S+:fÖo2NfA hyÐWcé‚Yu5nB)bf6[4–p5X,n;öA„;dYpX]–!yonW85*n;A'ldnÖßw`i '";1S[“=d?T‘MdAP=Äwq4GÜj]UK„” U2o;t”éobzneIQ

U`&:öDdPuctZ2QtSE%Ð r+XnEkàjSäöF,a*(m/FüymOdjY–ä]’C%„OßrA`éHÖRA,Gis’-à1;ÐGNvQVRE3G>üPB2.Y‘Ö,_/]öGg[stkHMDC,]BpP0J.–5SG:+?5–S:C–H‚1,‚YS ßULDLeu’hXQfjr/`kF]G2ÄfFlfvne!J*mM2YfGF0&c)ü”lXL„–BTIEyw2Vd'A4V6(glz"4j“e0/AsyHX8"hoöW<;4Ygrü.JARÜdö[[U8"[Äöa”L?'_R”ä55Ö


/usr/local/lib/python3.4/dist-packages/keras/models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


loss is
0.997184499908


--------------------------------------------------
Iteration 2
----- Generating with seed: "Didaktik des integrativen Unterrichts
Ab"
Didaktik des integrativen Unterrichts
Abb. 5.1: Überblick über die Stufen hinweg. Das heisst, es muss ein Kind haben wirklich und auch das mit dem Stoff drei Aufgaben zu einem für die Selbständigkeit zu erziehen. Dabei werden von den befragten Lehrpersonen auch die Frage der Integration von Kindern und Jugendlichen mit Schulschwierigkeiten integriert.
Schülerinnen und Schüler sollen die Lehrpersonen einen Lernraum eine gewisse Unabhängi
loss is
0.685830937905


--------------------------------------------------
Iteration 3
----- Generating with seed: "Didaktik des integrativen Unterrichts
Ab"
Didaktik des integrativen Unterrichts
Aber der Begegnung von Schülerinnen und Schülern mit Schulleistungsschwächen in der eigenen Entwicklung beitragen können. Die Kinder und Jugendlichen werden bereits in anderen Kontakt aufnehmen, um Bemüh

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:3: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


t auseinander. Wo grenzen wir uns ab, was können wir hier eine Vorstellung einer hinternehmen können, wir nehmen und kommen auch die anderen Schülerinnen und Schüler auf dem jeweiligen Niveau zu unterstützen. Das bedeutet wird, dass sie ausreichend Möglichkeiten zur Umsetzung von Integrativen Schulungs„indikig
loss is
0.525476150393


--------------------------------------------------
Iteration 7
----- Generating with seed: "Didaktik des integrativen Unterrichts
Ab"
Didaktik des integrativen Unterrichts
Aber dennoch: „Alles in allem ist der Unterricht auch an der (bspw. im Sinne einer Methodisierung von Unterricht als ‚Gestalten von Lernräumen‘ eingeführt. Unterstützungsmöglichkeiten für Lernprozesse sein. Allerdings verlier dieser Arbeit ist es mit der Schulischen Heilpädagogin oder dem H„Speutungsprozess beschrieben werden kann und einen anschliessenden Aktionsrahmen. Mit dieser Amtshandlung schl
loss is
0.498339390667


--------------------------------------------------
Iteration 9


In [20]:
print ('loss is')
print (history.history['loss'])
print()    
#model.save_weights('model_saved_weights')
#model.save('model_saved')
import pickle
pickle.dump([char_indices, indices_char], open( "15_chars.p", "wb" ))
!ls -rtlh

loss is
[0.4880149994876597]

total 103M
drwxr-xr-x 2 root root 4.0K Mar 20 09:50 textdatasets
-rw-r--r-- 1 root root  796 Mar 20 09:50 README.md
-rw-r--r-- 1 root root  13M Mar 21 15:09 model_saved_weights
-rw-r--r-- 1 root root  25M Mar 21 15:09 model_saved
-rw-r--r-- 1 root root  26K Mar 22 22:33 char_rnn_of_karpathy_keras.ipynb
-rw-r--r-- 1 root root 6.4K Mar 22 23:29 char_rnn_of_karpathy_keras_didactic.py
-rw-r--r-- 1 root root  14M Mar 23 10:41 Didactic_no_state_weights_1.h5
-rw------- 1 root root  18K Mar 23 10:42 nohup.out
-rw-r--r-- 1 root root  14M Mar 23 11:27 Didactic_no_state_weights_3.h5
-rw-r--r-- 1 root root 5.2K Mar 23 11:32 char_rnn_of_karpathy_keras_didactic_state_full.py
-rw-r--r-- 1 root root 4.8K Mar 23 11:32 nohup_state_full
-rw-r--r-- 1 root root  14M Mar 23 12:13 Didactic_no_state_weights_5.h5
-rw-r--r-- 1 root root  34K Mar 23 13:43 char_rnn_of_karpathy_keras_didactic-Copy1.ipynb
-rw-r--r-- 1 root root   72 Mar 23 13:44 Untitled.ipynb
-rw-r--r-- 1 root root  2